# Tasca S8.02. Power BI amb Python

Aquesta tasca consisteix en l'elaboració d'un informe de Power BI, aprofitant les capacitats analítiques de Python. S'utilitzaran els scripts de Python creats prèviament en la Tasca 1 per a generar visualitzacions personalitzades amb les biblioteques Seaborn i Matplotlib. Aquestes visualitzacions seran integrades en l'informe de Power BI per a oferir una comprensió més profunda de la capacitat del llenguatge de programació en l'eina Power BI.

Primer de tot he habilitat la creació de scripts de Python a Power BI.

Després fent clic a Obtenir dades, cerquem Script de Python i cliquem connectar. Aleshores enganxem el següent codi, utilitzat també en l'anterior Sprint 8.1, però amb algunes modificacions:

En primer lloc, he afegit 'collation' en la funció 'connection' per definir com es gestionen els caràcters i els ordres de les dades en la base de dades MySQL. D'aquesta manera s'assegura la comparació entre cadenes de caràcter i garanteix que les dades es puguin ordenar correctament. Estableixo la connexió amb la base de dades i executo la consulta 'SHOW TABLES' per obtenir els noms de totes les taules de la base de dades, tanco la connexió i faig que ens retorni els noms de les taules. A continuació executo una funció que obté els noms de les columnes de la taula, inicio un diccionari buit que anomeno 'dataframe', obtinc els noms de les taules i per cada taula, obtinc les dades i els noms de les columnes, converteixo les dades en un DataFrame de pandas i el guardo en el diccionari 'dataframe' amb el nom de la taula com a clau i faig que retorni el mateix. En últim lloc, executo la funció 'fill_data()' perquè obtingui les dades de totes les taules i les emmagatzemi en el diccionari i assigno cada DataFrame a una variable específica amb el nom de la taula. En aquest cas, no he carregat ni 'credit_cards' ni 'credit_cards_status' perquè no les faig servir en cap visualització.

In [ ]:
# Importem les llibreries i mòduls
import pandas as pd
import mysql.connector

def connection():
    # Connexió amb la base de dades MySQL mitjançant les credencials
    connection = mysql.connector.connect(
        host="localhost",
        user="maria",
        password="marmota",
        database="test",
        collation="utf8mb4_unicode_ci" 
    )
    return connection

# Funció per obtenir els noms de les taules
def getTableNames():
    conn = connection()
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    results = cursor.fetchall()
    cursor.close()
    conn.close()
    return results

# Funció per obtenir dades d'una taula amb els seus noms de columnes
def getTableWithColumns(table):
    conn = connection()
    cursor = conn.cursor()
    cursor.execute(f'SELECT * FROM {table}')
    results = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    cursor.close()
    conn.close()
    return results, columns

# Funció per omplir un diccionari amb dataframes per cada taula
def fill_data():
    dataframe = {}
    tables = getTableNames()
    for table in tables:
        rows, columns = getTableWithColumns(table[0])
        dataframe[table[0]] = pd.DataFrame(rows, columns=columns)
    return dataframe

# Obtenir les dades de les taules
dataframe = fill_data()

users = dataframe['users']
products= dataframe['products']
transactions_products= dataframe['transactions_products']
transactions= dataframe['transactions']
companies = dataframe['companies']

El dataframe es carrega correctament a Power BI, amb les taules que necessitaré per generar totes les visualitzacions: 'companies', 'products', 'transactions', 'transactions_products' i 'users'.

Abans de procedir a fer les visualitzacions, accedeixo a la Vista de model i estableixo les relacions entre les taules. També em demanarà d'habilitar els objectes visuals de script una vegada vulgui crear el primer objecte visual de Python.

## NIVELL 1

Els 7 exercicis del nivell 1 de la tasca 01

## Exercici 1

Una vegada seleccionat l'objecte visual de Python, cal afegir els valors 'weight' i 'id' de  la taula 'products', sense resumir. Afegeixo l'identificador perquè el recompte de les quantitats aparegui de manera correcta. També he fet canvis en els valors de 'weight' i 'price', que necessitaré després, ja que penso que per un error de lectura del punt flotant, els valors numèrics no s'han carregat a Power BI amb l'escala correcte. Com a solució temporal, i no la més aconsellable, he dividit els valors per 100 per tal de corregir-los. Però entenc que cal buscar quin és el motiu d'aquest problema (configuracions de tipus de dades, configuracions regionals, opcions del connector...). Després he posat el tipus de format decimial i la moneda en €. 

Una vegada fet tot això, ja podem enganxar l'script utilitzat en l'exercici 1 i que mostro a continuació, també amb alguns canvis. Per una banda, he necessitat carregar les llibreries cada vegada que les utilitzava en el codi, he substituït 'dataframe' per 'dataset' en tot el codi, que és la terminologia pròpia de Power BI per referir-se al conjunt de dades. I he tret la part de codi que no s'executava, com és el print final que faig per mostrar els stats amb la mitjana, la desviació estàndard, etc., que en el ipynb podíem veure, però com és lògic, en el dashboard de Power BI no es visualitzarà. També he tret aquesta línia de codi que grega la moda, mediana i rang a les estadístiques descriptives:

stats['mode'] = round(mode_weight, 2)
stats['median'] = round(median_weight, 2)
stats['range'] = round(range_weight, 2)

A continuació, mostro el codi executat en l'editor de scripts de Python:

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.dataframe(weight, id)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

# Importar les llibreries
import matplotlib.pyplot as plt
import seaborn as sns

# Obtenir estadístiques descriptives
stats = dataset['weight'].describe().round(2)
mean_weight = stats['mean']
std_weight = stats['std']

# Calcular la moda, la mediana i el rang
mode_weight = dataset['weight'].mode()[0]  # La moda pot tenir més d'un valor, seleccionem el primer
median_weight = dataset['weight'].median()
range_weight = dataset['weight'].max() - dataset['weight'].min()

# Seleccionar un estil de Matplotlib
plt.style.use('bmh')

# Comprova si la columna 'weight' existeix
if 'weight' in dataset.columns:

    # Crear la figura
    plt.figure(figsize=(12, 5))

    # Crear l'histograma
    sns.histplot(data=dataset, x="weight", bins=30, kde=True, color='royalblue', edgecolor='black')

    # Afegir línies verticals per a la mitjana i la desviació estàndard
    plt.axvline(mean_weight, color='red', linestyle='dashed', linewidth=2, label=f'Mitjana: {mean_weight:.2f}')
    plt.axvline(mean_weight + std_weight, color='orange', linestyle='dashed', linewidth=2, label=f'+1 STD: {mean_weight + std_weight:.2f}')
    plt.axvline(mean_weight - std_weight, color='orange', linestyle='dashed', linewidth=2, label=f'-1 STD: {mean_weight - std_weight:.2f}')

    # Configurar les etiquetes i el títol dels eixos
    plt.title('Distribució del pes dels productes', fontsize=16)
    plt.xlabel('Pes (Kilograms)')
    plt.ylabel('Quantitat')

    # Modificar les etiquetes de l'eix Y per intervals específics 
    plt.yticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

    # Col·locar la llegenda a la part superior dreta
    plt.legend(loc='upper right')

    # Mostrar la visualització
    plt.show()

## Exercici 2





Afegim 'weight' i 'price' a valors, enganxem el codi i fem el canvi de dataframe a dataset.

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.dataframe(weight, price)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

# Importar les llibreries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 

# Calcular la correlació 
correlation = dataset['weight'].corr(dataset['price'])

# Crear una nova columna que classifica el pes per intervals
dataset['weight_range'] = pd.cut(dataset['weight'], bins=[0.6, 1.0, 1.5, 2.0, 2.5, 3.0, 3.2])

# Només es mostren les combinacions de categories que tenen dades
grouped = dataset.groupby('weight_range', observed=True)['price'].mean()

# Verificar si les columnas 'price' i 'weight' existeixen en el DataFrame
if 'price' in dataset.columns and 'weight' in dataset.columns:

    # Convertir les columnes a float
    dataset['price'] = dataset['price'].astype(float)
    dataset['weight'] = dataset['weight'].astype(float)

    # Agregar una columna pels rangs de pes
    bins = [0.60, 1.00, 1.50, 2.00, 2.50, 3.00, 3.20]
    labels = ['0.60-1.00', '1.01-1.50', '1.51-2.00', '2.01-2.50', '2.51-3.00', '3.01-3.20']
    dataset['weight_range'] = pd.cut(dataset['weight'], bins=bins, labels=labels, right=False)

    # Seleccionar un estil de Matplotlib
    plt.style.use('bmh')

    # Crear la figura
    plt.figure(figsize=(12, 8))

    # Crear el gràfic de dispersió amb una línia de regressió
    ax = sns.scatterplot(x=dataset['price'], y=dataset['weight'], hue=dataset['weight_range'], palette='viridis')
    sns.regplot(x='price', y='weight', data=dataset, scatter=False, ax=ax, color='royalblue')

    # Configurar les etiquetes dels eixos i el títol
    plt.title('Relació entre Preu i Pes dels Productes', fontsize=16)
    plt.xlabel('Preu (€)')
    plt.ylabel('Pes (kg)')

    # Establir els límits de l'eix Y perquè comenci a 0
    plt.ylim(0, dataset['weight'].max() + 0.5)
    plt.xlim(0)

    # Col·locar la llegenda
    plt.legend(title='Rangs de Pes', loc='upper right')

    # Afegir un marge al voltant de la figura
    plt.gcf().set_facecolor('white')
    plt.gcf().tight_layout(pad=3.5)

    # Mostrar el gràfic
    plt.show()

## Exercici 3

En aquest exercici, he canviat la part del codi de 'count' per 'id' i des dels valors de la visualització que fes un recompte distintiu dels valors de la variable. Finalment, també he agregat la funció sort_values de pandas, per ordenar les barres de major a menor quantitat. També he tret tota la part final on faig els càlculs dels estadístics descriptius. El codi queda així:

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(colour, id)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:
import matplotlib.pyplot as plt
import pandas as pd

# Establir l'estil del gràfic 
plt.style.use('ggplot')  

# Ordenar el DataFrame per la columna 'id' de major a menor
dataset_sorted = dataset.sort_values(by='id', ascending=False)

# Crear el gràfic de barres amb colors personalitzats
plt.figure(figsize=(16, 10))
bars = plt.bar(dataset_sorted['colour'], dataset_sorted['id'], color=dataset_sorted['colour'], edgecolor='black')

# Afegir etiquetes i títol
plt.xlabel('Gamma de Colors')
plt.ylabel('Quantitat')
plt.title('Quantitat de Productes per Color', fontsize=20)

# Configurar les etiquetes de l'eix Y sense decimals
plt.yticks(range(0, int(dataset_sorted['id'].max()) + 1))

# Ocultar les etiquetes de l'eix X
plt.xticks([])

# Mostrar el gràfic
plt.show()

## Exercici 4

Per aquesta visualització, utilitzem 'country' de companies i 'amount' sense resumir i faig els mateixos canvis que ja assenyalo en exercicis anteriors.

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(country, amount)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Establir l'estil del gràfic 
plt.style.use('ggplot')  

# Imprimir el recompte de transaccions per país
transaction_counts = dataset.groupby('country').size()
print(transaction_counts)

# Calcular la desviació estàndard per país
std_devs = dataset.groupby('country')['amount'].std()

# Definir els llindars per categoritzar la variabilitat
low_threshold = std_devs.quantile(0.33)
high_threshold = std_devs.quantile(0.66)

# Crear una nova columna per a les categories de variabilitat
dataset['variability'] = dataset['country'].apply(lambda x: 'Baixa' if std_devs[x] <= low_threshold else ('Mitja' if std_devs[x] <= high_threshold else 'Alta'))

# Definir la paleta de colors
palette = {'Baixa': 'lightgreen', 'Mitja': 'khaki', 'Alta': 'salmon'}

# Crear el boxplot amb colors personalitzats
plt.figure(figsize=(15, 8))

# Assegurar-se que els països s'ordenen correctament en el DataFrame
order = dataset['country'].unique()
ax = sns.boxplot(x='country', y='amount', hue='variability', data=dataset, order=order, palette=palette, dodge=False)

# Afegir etiquetes de mediana al boxplot
medians = dataset.groupby('country')['amount'].median()
for tick, label in enumerate(order):
    plt.text(tick, medians[label] + 0.5, f"{medians[label]:.2f}", 
             horizontalalignment='center', size='medium', color='black')

# Configurar etiquetes dels eixos i títol
plt.title('Import de les Transaccions per País', fontsize=20)
plt.xlabel('País')
plt.ylabel('Import de les transaccions (€)')

# Rotar les etiquetes de l'eix X per millorar la llegibilitat
plt.xticks(rotation=45)

# Afegir la llegenda a la part superior dreta
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles=handles, labels=['Baixa', 'Mitja', 'Alta'], title='Variabilitat', loc='upper left')

# Mostrar el gràfic
plt.show()

## Exercici 5

En aquesta visualització, torno a utilitzar 'country' de 'companies' i creo aquesta nova columna perquè ens faci el recompte:

declined_count = 
CALCULATE(
    COUNTROWS(transactions),
    transactions[declined] = 1
)

El codi queda de la següent manera:

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(country, declined_count)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import matplotlib.pyplot as plt
import numpy as np  

# Estil del gràfic
plt.style.use('ggplot')  

# Calcular el valor mitjà de transaccions declinades
mean_declined = dataset['declined_count'].mean()

# Ordenar els països per número de transaccions declinades
country_counts_sorted = dataset.sort_values(by='declined_count', ascending=False).reset_index(drop=True)

# Crear el gràfic
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': 'polar'})

# Calcular l'angle per cada país
angles = np.linspace(0, 2 * np.pi, len(country_counts_sorted), endpoint=False).tolist()

# Plotejar les línies del gràfic
values = country_counts_sorted['declined_count'].tolist()
values += values[:1]  
angles += angles[:1]  

ax.plot(angles, values, 'o-', linewidth=2)
ax.fill(angles, values, alpha=0.25)

# Etiquetar els països
ax.set_yticklabels([])
ax.set_xticks(angles[:-1])
ax.set_xticklabels(country_counts_sorted['country'], size=10)

# Configurar el títol i mostrar el gràfic
plt.title('Diagrama de Xarxa de les Transaccions Declinades per País', size=20)
plt.show()

## Exercici 6

Per la visualització del mapa de calor, prèviament creo la columna 'age' amb DAX:

age = DATEDIFF('users'[birth_date], TODAY(), YEAR)

Després, agrego a valors la mitjana de 'amount', 'country' de 'users' i 'age'. 

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount, country, age)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import matplotlib.pyplot as plt
import seaborn as sns  

# Agrupar per país i edat, i calcular la mitjana de l'amount
grouped_dataset = dataset.groupby(['country', 'age']).agg({'amount': 'mean'}).reset_index()

# Crear una taula pivote pel heatmap
pivot_dataset = grouped_dataset.pivot(index='age', columns='country', values='amount').fillna(0)

# Crear el heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_dataset, cmap="YlGnBu", annot=True, fmt=".2f", linewidths=.5)

# Afegir el símbol de € a les anotacions
for text in plt.gca().texts:
    value = text.get_text()
    if value != 'nan':
        text.set_text(f'{value}€')

plt.title('Mitjana Import de les Transaccions per Edat i País', size=20)
plt.xlabel('País')
plt.ylabel('Edat')
plt.show()

## Exercici 7

Per graficar el pairplot, hem afegit els valors 'country' de la taula 'users', 'weight', 'price' i el promig de l'amount. El pairplot no es mostra de la mateixa manera a Power BI, ja que la diagonal no es genera el gràfic correctament. 

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(country, amount, weight, price)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:
import matplotlib.pyplot as plt
import seaborn as sns

# Establir l'estil del gràfic 
plt.style.use('ggplot')  

# Calcular la mitjana de l'amount per país
dataset_avg_amount = dataset.groupby('country')['amount'].mean().reset_index()
dataset_avg_amount.rename(columns={'amount': 'average_amount'}, inplace=True)

# Funció per obtenir una mostra aleatòria de 62 registres per país
def get_random_sample(dataset, sample_size):
    return dataset.sample(min(len(dataset), sample_size))

# Aplicar la funció per obtenir una mostra aleatòria amb 62 registres per país
dataset_sampled = dataset.groupby('country', group_keys=False).apply(lambda x: get_random_sample(x, 62))

# Unir la mitjana de l'amount amb les altres dades
dataset_sampled = dataset_sampled.merge(dataset_avg_amount, on='country')

# Triar les variables seleccionades per al pairplot
selected_vars = ['average_amount', 'weight', 'price']

# Configurar el gràfic
plt.figure(figsize=(14, 10))  # Ajustar la mida de la figura

# Crear el pairplot amb la tercera variable
pairplot = sns.pairplot(dataset_sampled, vars=selected_vars, height=4, aspect=1, hue='country', palette='husl')

# Afegir un títol al gràfic
pairplot.fig.suptitle("Mitjana de l'import, el pes i els preus dels productes pels diferents països", y=1, fontsize=24)

# Ajustar la posició de la llegenda i el títol
pairplot.fig.subplots_adjust(top=0.92)    

# Mostrar el gràfic
plt.show()

## NIVELL 2

Els 2 exercicis del nivell 2 de la tasca 01

## Exercici 1

Per integrar el gràfic dispersió tridimensional, he afegit els valors 'weight', 'price' i 'amount'. 

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(weight, price, amount)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

# Importar les llibreries

import matplotlib.pyplot as plt

# Convertir les columnes numèriques a tipus float per evitar errors
dataset['weight'] = dataset['weight'].astype(float)
dataset['price'] = dataset['price'].astype(float)
dataset['amount'] = dataset['amount'].astype(float)

# Crear la figura 3D
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

# Extreure les variables per a l'eix x, y, z
x = dataset['weight']
y = dataset['price']
z = dataset['amount']

# Crear el gràfic de dispersió 3D
sc = ax.scatter(x, y, z, c='blue', marker='o')

# Etiquetar els eixos
ax.set_xlabel('Weight')
ax.set_ylabel('Price')
ax.set_zlabel('Amount')

# Afegir un títol
ax.set_title('Correlació entre Pes, Preu i Import', fontsize=24)

# Mostrar el gràfic
plt.show()

## Exercici 2

Per fer el jointplot, abans he creat una nova columna calculada a la taula 'transactions_products' anomenada 'quantity':

quantity = COUNTROWS(transactions_products)

Després he afegit les tres variables, 'country' de 'users', 'amount' i el recompte de 'quantity', i executat el següent codi a l'editor.

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(country, amount, quantity)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import matplotlib.pyplot as plt
import seaborn as sns

# Estil del gràfic
plt.style.use('ggplot')  

# Configurar el gràfic de Seaborn
plt.figure(figsize=(12, 8))

# Crear el jointplot amb les variables seleccionades
joint_plot = sns.jointplot(x='amount', y='quantity', data=dataset, kind='scatter', hue='country', height=8, ratio=5, palette='husl')

# Ajustar els límits de l'eix X
joint_plot.ax_joint.set_xlim(-100, 600)

# Afegir títol
plt.suptitle('Import i Quantitat de productes agrupat per País', y=1, fontsize=20)

# Afegir marges blancs al voltant de la figura
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

# Mostrar el gràfic
plt.show()

## NIVELL 3

Els 2 exercicis del nivell 3 de la tasca 01

## Exercici 1

Per visualitzar el violinplot he tornat a utilitzar 'country' de 'users', 'amount' i el recompte de 'quantity'.

Veiem el codi complet:

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(country, amount, quantity)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import matplotlib.pyplot as plt
import seaborn as sns

# Configurar la mida de la figura
plt.figure(figsize=(12, 6))

# Crear un grid de subplots para combinar els gràfics
plt.subplot(1, 2, 1)  # Primer subplot pel Violinplot
sns.violinplot(x='country', y='quantity', data=dataset, palette='husl')
plt.title('Quantitat de Productes per País')
plt.xlabel('País')
plt.ylabel('Quantitat')

plt.subplot(1, 2, 2)  # Segon subplot pel gràfic de barres
sns.barplot(x='country', y='amount', data=dataset, estimator=sum, ci=None, palette='husl')
plt.title('Suma Import per País')
plt.xlabel('País')
plt.ylabel('Suma Import (€)')

# Ajustar els límits de l'eix Y
plt.ylim(0, dataset['amount'].sum() + 100)  # Ajustar el límit de l'eix Y
    
# Ajustar el layout dels subplots
plt.tight_layout()

# Mostrar el gràfic
plt.show()

## Exercici 2

Per fer a l'última visualització, he utilitzat 'country', 'amount' i el recompte de 'quantity' de la taula 'transactions_products'. També he realitzat algunes modificacions en codi, perquè m'apareixien diferents errors, atribuïts al Facet Grid, per exemple aquest:

On tenia... 

g.map_dataset(barplot_with_highlight, x='product_id', y='id', order=dataset['product_id'].unique())

I ho he canviat per:

g.map(barplot_with_highlight, 'product_id', 'id')

A continuació el codi complet:

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(users, product_id, quantity)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Convertir les columnes numèriques a tipus float/int per evitar errors
dataset['quantity'] = dataset['quantity'].astype(int)

# Ordenar el DataFrame per product_id
dataset = dataset.sort_values(by='product_id')

# Estil del gràfic
plt.style.use('ggplot')  

# Configurar la mida global de la figura
plt.figure(figsize=(18, 12)) 

# Configurar el gràfic de Seaborn
g = sns.FacetGrid(dataset, col='country', col_wrap=3, height=6) 

# Funció per pintar les barres
def barplot_with_highlight(x, y, **kwargs):
    ax = sns.barplot(x=x, y=y, **kwargs)
    data = kwargs['data']
    # Identificar els  productes més venuts
    top5 = data.nlargest(5, 'quantity')
    most_sold = data.nlargest(1, 'quantity')
    least_sold = data.nsmallest(1, 'quantity')
    # Pintar les barres dels colors
    for bar, product_id in zip(ax.patches, data['product_id']):
        if product_id in most_sold['product_id'].values:
            bar.set_color('seagreen')
        elif product_id in least_sold['product_id'].values:
            bar.set_color('r')
        elif product_id in top5['product_id'].values:
            bar.set_color('sandybrown')
        else:
            bar.set_color('dodgerblue')
    # Afegir la línia de la mitjana
    mean_value = data['quantity'].mean()
    ax.axhline(mean_value, color='black', linestyle='--')
    ax.text(0, mean_value, f'Media: {mean_value:.2f}', color='black', ha='right')
    return ax

# Mapejar el gràfic de barres per cada subplot amb barplot_with_highlight
g.map_dataframe(barplot_with_highlight, x='product_id', y='quantity')

# Ajustar l'escala de l'eix y
for ax in g.axes.flat:
    ax.set_ylim(0, 50)  

# Rotar les etiquetes de l'eix x
for ax in g.axes.flat:
    ax.set_xticks(np.arange(len(dataset['product_id'].unique())))
    ax.set_xticklabels(dataset['product_id'].unique(), rotation=90)

# Afegir el títol als subpolts
g.set_titles('{col_name}')

# Afegir el títol general
g.fig.suptitle("Ventas de producto por país de usuario", y=1, fontsize=20)

# Afegir etiquetes als eixos
g.set_axis_labels(x_var='ID Productos', y_var='Unidades vendidas')

# Ajustar l'espai entre els subplots
plt.subplots_adjust(left=0.05, right=0.95, top=0.9, bottom=0.1)

# Mostrar el gràfic
plt.show()